<a href="https://colab.research.google.com/github/AnupJoseph/adv-python/blob/master/Concurrency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aiohttp

     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 266kB 16.2MB/s 
     |████████████████████████████████| 153kB 18.8MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=770972198bfd03c8891fb3fdd8edd3dde8edaee621c3a67ba406c0ef7f5f2f75
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [ ]:
# I/O Bound operations speedup

In [2]:
import requests
import time
import concurrent.futures
import threading
import asyncio
import aiohttp
import multiprocessing

In [3]:
# Non-concurrent program to download 160 sites
def download_site(url,session):
  with session.get(url) as response:
    print(f"Read {len(response.content)} from {url}")

def download_all_sites(sites):
  with requests.Session() as session:
    for url in sites:
      download_site(url,session)

if __name__ == "__main__":
  sites = [
    "https://www.jython.org",
    "http://olympus.realpython.org/dice"
  ]*80
  start_time = time.time()
  download_all_sites(sites)
  duration = time.time() - start_time
  print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jyth

In [4]:
thread_local = threading.local()

def get_session():
  if not hasattr(thread_local,"session"):
    thread_local_session = requests.Session()
  return thread_local_session

def download_site(url):
  session = get_session()
  with session.get(url) as response:
    print(f"Read {len(response.content)} from {url}")

def download_all_sites(sites):
  with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_site,sites)

if __name__ == "__main__":
  sites = [
    "https://www.jython.org",
    "http://olympus.realpython.org/dice"
  ]*80
  start_time = time.time()
  download_all_sites(sites)
  duration = time.time() - start_time
  print(f"Downloaded {len(sites)} in {duration} seconds")

Read 275 from http://olympus.realpython.org/diceRead 275 from http://olympus.realpython.org/dice

Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 275 from http://olympus.realpython.org/diceRead 10267 from https://www.jython.org

Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https:

In [ ]:
# Asyncio version has some unfixed erros
async def download_site(session, url):
    async with session.get(url) as response:
        print("Read {0} from {1}".format(response.content_length, url))


async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")

In [6]:
# Concurrent version
session = None

def set_global_session():
  global session
  if not session:
    session = requests.Session()

def download_site(url):
  with session.get(url) as response:
    name = multiprocessing.current_process().name
    print(f"{name}:Read {len(response.content)} from {url}")

def download_all_sites(sites):
  with multiprocessing.Pool(initializer = set_global_session) as pool:
    pool.map(download_site, sites)

if __name__ == "__main__":
  sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
  ] * 80
  start_time = time.time()
  download_all_sites(sites)
  duration = time.time() - start_time
  print(f"Downloaded {len(sites)} in {duration} seconds")

ForkPoolWorker-1:Read 10267 from https://www.jython.org
ForkPoolWorker-2:Read 10267 from https://www.jython.org
ForkPoolWorker-1:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10267 from https://www.jython.org
ForkPoolWorker-2:Read 10267 from https://www.jython.org
ForkPoolWorker-1:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10267 from https://www.jython.org
ForkPoolWorker-2:Read 10267 from https://www.jython.org
ForkPoolWorker-1:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10267 from https://www.jython.org
ForkPoolWorker-2:Read 10267 from https://www.jython.org
ForkPoolWorker-1:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 275 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10

In [ ]:
# CPU bound execution speedup

In [11]:
def cpu_bound(number):
  return sum(i*i for i in range(number))

def find_sums(numbers):
  for number in numbers:
    cpu_bound(number)

if __name__ == "__main__":
  numbers = [5_000_000+x for x in range(20)]

  start_time = time.time()
  find_sums(numbers)
  duration = time.time() - start_time
  print(f"Duration {duration} seconds")

Duration 9.543235063552856 seconds


In [12]:
def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration = time.time() - start_time
    print(f"Duration {duration} seconds")

Duration 10.451621770858765 seconds
